## USEFULL LINKS

CryptoCompare API Documentation: https://min-api.cryptocompare.com/documentation

REFERENCE ARTICLE: https://blog.cryptocompare.com/cryptocompare-api-quick-start-guide-f7abbd20d260

GetOldTweets Library: https://pypi.org/project/GetOldTweets3/



## LIBRARIES

In [1]:
import requests
import GetOldTweets3 as got
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import dateutil.relativedelta

## FUNCTIONS

#### PRICE DATA

In [2]:
def get_btc_price(date):
    """ Query the API for 2000 hourly historical price points starting from "date". date in unix"""
    crypto = "BTC" # Cryptocurrency 
    fiat = "EUR" # Fiat currency
    lmt = str(2000) # query limit in string format, max = 2000
    
    api_key = "5777fd1cfc0edbbc82cefe1b4d616d9fe9a4e3a8cd32045d53d1a31e7e7a7454"
    url = f"https://min-api.cryptocompare.com/data/v2/histohour?fsym={crypto}&tsym={fiat}&limit={lmt}&toTs={date}&api_key={api_key}"
    
    r = requests.get(url)
    ipdata = r.json()
    return ipdata


def get_df_price(from_date, to_date):
    """ Get historical price data between two dates. dates in unix """
    date = to_date
    holder = []
    # While the earliest date returned is later than the earliest date requested, keep on 
    # querying the API and adding the results to a list. 
    while date > from_date:
        data = get_btc_price(date)["Data"]
        holder.append(pd.DataFrame(data["Data"]))
        date = data['TimeFrom']
    # Join together all of the API queries in the list.    
    df = pd.concat(holder, axis = 0)                    
    # Remove data points from before from_date
    df = df[df['time']>from_date]                       
    # Convert to timestamp to readable date format
    df['time'] = pd.to_datetime(df['time'], unit='s')   
    # Make the DataFrame index the time
    df.set_index('time', inplace=True)                  
    # And sort it so its in time order 
    df.sort_index(ascending=False, inplace=True)        
    return df

#### SOCIAL DATA

In [3]:
def get_btc_soc(date):
    """ Query the API for 2000 historical social data points starting from "date". date in unix"""
    crypto = 1182 # Cryptocurrency ID (BTC here) 
    lmt = str(2000) # query limit in string format, max = 2000
    
    api_key = "5777fd1cfc0edbbc82cefe1b4d616d9fe9a4e3a8cd32045d53d1a31e7e7a7454"
    url = f"https://min-api.cryptocompare.com/data/social/coin/histo/hour?coinId={crypto}&limit={lmt}&toTs={date}&api_key={api_key}"
    
    r = requests.get(url)
    ipdata = r.json()
    return ipdata


def get_df_soc(from_date, to_date):
    """ Get historical social data between two dates. dates in unix """
    date = to_date
    holder = []
    # While the earliest date returned is later than the earliest date requested, keep on 
    # querying the API and adding the results to a list. 
    while date > from_date:
        data = get_btc_soc(date)["Data"] #access Data key of returned dictionary
        holder.append(pd.DataFrame(data))
        date = data[0]["time"]
    # Join together all of the API queries in the list.    
    df = pd.concat(holder, axis = 0)                    
    # Remove data points from before from_date
    df = df[df['time']>from_date]                       
    # Convert to timestamp to readable date format
    df['time'] = pd.to_datetime(df['time'], unit='s')   
    # Make the DataFrame index the time
    df.set_index('time', inplace=True)                  
    # And sort it so its in time order 
    df.sort_index(ascending=False, inplace=True)        
    return df

#### TWEETS

In [4]:
def get_btc_tweets(since, until):
    """ Query the GetOldTweets Library for given number of tweets during specified timeframe
    and returns DataFrame. Date in "%Y-%m-%d" format. If number of tweets is 
    high short timeframe is recomended (i.e. 1 day))"""
    crypto = "bitcoin" # Cryptocurrency ID (BTC here)
    keyword = "trend"
    lmt = 2400 # number of tweets limit
    top_only = False
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(f"{crypto} {keyword}")\
                                           .setSince(since)\
                                           .setUntil(until)\
                                           .setMaxTweets(lmt)
                                           
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    
    time = [pd.to_datetime(datetime.timestamp(t.date),unit="s").replace(minute=0, second=0)\
        for t in tweets]
    text = [t.text for t in tweets]
    retweets = [t.retweets for t in tweets]
    ipdata = {"time" : time, "text" : text, "retweets" : retweets}
    ipdata = pd.DataFrame(ipdata)
    ipdata.drop_duplicates(subset="text", keep='first',inplace=True)
    ipdata.reset_index(drop=True, inplace=True)
    return ipdata


def get_df_tweets(from_date, to_date):
    """ Loops the Query for GetOldTweets Library for given tweets over specified timeframe,
    saves each iteration in a CSV and returns DataFrame. Date in datetime format"""
    since = to_date - dateutil.relativedelta.relativedelta(days=1)
    until = to_date
    
    holder = []
    # While the earliest date returned is later than the earliest date requested, keep on 
    # querying the GOT Library and adding the results to a list. 
    while since > from_date:
        since = since.strftime("%Y-%m-%d")
        until = until.strftime("%Y-%m-%d")
        data = get_btc_tweets(since, until)
        holder.append(data)
        until = data["time"][data.index[-1]]
        since = data["time"][data.index[-1]]- dateutil.relativedelta.relativedelta(days=1)
    # Join together all of the results of queries in the list.    
    df = pd.concat(holder, axis = 0)                    
    # Remove data points from before from_date
    df = df[df['time']>from_date]
    # Reset index of final dataframe
    df.reset_index(drop=True, inplace=True)
    return df

## DATA COLLECTION

### BITCOIN PRICE

In [5]:
#Hourly DATA FOR THIS TIME HORIZON:
start_date = int(datetime.timestamp(datetime(2017, 7, 1, 0, 0,0)))
end_date = int(datetime.timestamp(datetime.today())) 

In [6]:
btc_price = get_df_price(start_date,end_date)
btc_price.head()

,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
time,,,,,,,,
2019-12-16 10:00:00,6343.81,6336.91,6341.45,108.28,686535.97,6339.78,direct,
2019-12-16 09:00:00,6352.29,6336.02,6346.54,368.97,2340380.72,6341.45,direct,
2019-12-16 08:00:00,6354.45,6332.84,6342.16,349.59,2217356.03,6346.54,direct,
2019-12-16 07:00:00,6362.94,6339.12,6356.61,232.95,1479130.90,6342.16,direct,
2019-12-16 06:00:00,6359.02,6344.38,6344.85,92.17,585764.06,6356.61,direct,


In [7]:
btc_price.shape

(20829, 8)

In [8]:
btc_price.drop(["conversionType","conversionSymbol"],axis=1,inplace=True)
btc_price.head()

,high,low,open,volumefrom,volumeto,close
time,,,,,,
2019-12-16 10:00:00,6343.81,6336.91,6341.45,108.28,686535.97,6339.78
2019-12-16 09:00:00,6352.29,6336.02,6346.54,368.97,2340380.72,6341.45
2019-12-16 08:00:00,6354.45,6332.84,6342.16,349.59,2217356.03,6346.54
2019-12-16 07:00:00,6362.94,6339.12,6356.61,232.95,1479130.90,6342.16
2019-12-16 06:00:00,6359.02,6344.38,6344.85,92.17,585764.06,6356.61


In [9]:
#check types and nulls:
for col in btc_price.columns:
    print(col, "  TYPE:", btc_price[col].dtype, "  NULLS", btc_price[col].isna().sum())

high   TYPE: float64   NULLS 0
low   TYPE: float64   NULLS 0
open   TYPE: float64   NULLS 0
volumefrom   TYPE: float64   NULLS 0
volumeto   TYPE: float64   NULLS 0
close   TYPE: float64   NULLS 0


### BITCOIN VOLUME

### BITCOIN SOCIAL

In [10]:
btc_soc = get_df_soc(start_date, end_date)
btc_soc.head()

/home/mattia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,analysis_page_views,charts_page_views,code_repo_closed_issues,code_repo_closed_pull_issues,code_repo_contributors,code_repo_forks,code_repo_open_issues,code_repo_open_pull_issues,code_repo_stars,code_repo_subscribers,...,reddit_posts_per_day,reddit_posts_per_hour,reddit_subscribers,total_page_views,trades_page_views,twitter_favourites,twitter_followers,twitter_following,twitter_lists,twitter_statuses
time,,,,,,,,,,,,,,,,,,,,,
2019-12-16 10:00:00,1052737,8776837,4368,12021,829,24795,765,322,41571,3505,...,73.30,3.05,1215529,47766522,767349,1000,844049,165,6631,20316
2019-12-16 09:00:00,1052733,8776769,4368,12021,829,24794,765,321,41570,3505,...,73.21,3.05,1215492,47765916,767344,1000,844049,165,6631,20316
2019-12-16 08:00:00,1052718,8776654,4368,12021,829,24794,765,321,41569,3505,...,71.29,2.97,1215465,47765308,767339,1000,844049,165,6631,20316
2019-12-16 07:00:00,1052712,8776584,4368,12021,829,24794,765,321,41568,3505,...,72.62,3.03,1215443,47764764,767335,1000,844049,165,6631,20316
2019-12-16 06:00:00,1052707,8776532,4368,12021,829,24793,765,321,41567,3505,...,71.98,3.00,1215371,47764243,767333,1000,844049,165,6631,20316


In [11]:
btc_soc.shape

(20829, 33)

In [12]:
#check types and nulls and DUPLICATES:
for col in btc_soc.columns:
    print(col, "  TYPE:", btc_soc[col].dtype, "  NULLS", btc_soc[col].isna().sum())

analysis_page_views   TYPE: int64   NULLS 0
charts_page_views   TYPE: int64   NULLS 0
code_repo_closed_issues   TYPE: int64   NULLS 0
code_repo_closed_pull_issues   TYPE: int64   NULLS 0
code_repo_contributors   TYPE: int64   NULLS 0
code_repo_forks   TYPE: int64   NULLS 0
code_repo_open_issues   TYPE: int64   NULLS 0
code_repo_open_pull_issues   TYPE: int64   NULLS 0
code_repo_stars   TYPE: int64   NULLS 0
code_repo_subscribers   TYPE: int64   NULLS 0
comments   TYPE: int64   NULLS 0
fb_likes   TYPE: int64   NULLS 0
fb_talking_about   TYPE: int64   NULLS 0
followers   TYPE: int64   NULLS 0
forum_page_views   TYPE: int64   NULLS 0
influence_page_views   TYPE: int64   NULLS 0
markets_page_views   TYPE: int64   NULLS 0
overview_page_views   TYPE: int64   NULLS 0
points   TYPE: int64   NULLS 0
posts   TYPE: int64   NULLS 0
reddit_active_users   TYPE: int64   NULLS 0
reddit_comments_per_day   TYPE: float64   NULLS 0
reddit_comments_per_hour   TYPE: float64   NULLS 0
reddit_posts_per_day   

In [13]:
#check for non null, but absent hourly data
for col in btc_soc.columns:
    print(col, "  ZEROS:", len(btc_soc.loc[btc_soc[col]== 0]))

analysis_page_views   ZEROS: 12438
charts_page_views   ZEROS: 12438
code_repo_closed_issues   ZEROS: 12441
code_repo_closed_pull_issues   ZEROS: 12438
code_repo_contributors   ZEROS: 18890
code_repo_forks   ZEROS: 12438
code_repo_open_issues   ZEROS: 12438
code_repo_open_pull_issues   ZEROS: 12439
code_repo_stars   ZEROS: 12438
code_repo_subscribers   ZEROS: 12438
comments   ZEROS: 12438
fb_likes   ZEROS: 12438
fb_talking_about   ZEROS: 12978
followers   ZEROS: 12438
forum_page_views   ZEROS: 12438
influence_page_views   ZEROS: 12438
markets_page_views   ZEROS: 12438
overview_page_views   ZEROS: 12438
points   ZEROS: 12438
posts   ZEROS: 12438
reddit_active_users   ZEROS: 12438
reddit_comments_per_day   ZEROS: 12438
reddit_comments_per_hour   ZEROS: 12438
reddit_posts_per_day   ZEROS: 12438
reddit_posts_per_hour   ZEROS: 12438
reddit_subscribers   ZEROS: 12438
total_page_views   ZEROS: 12438
trades_page_views   ZEROS: 12438
twitter_favourites   ZEROS: 12438
twitter_followers   ZEROS: 1

In [14]:
#check closest time for which at least 1 col with data != 0:
btc_soc[btc_soc["twitter_statuses"] != 0]


,analysis_page_views,charts_page_views,code_repo_closed_issues,code_repo_closed_pull_issues,code_repo_contributors,code_repo_forks,code_repo_open_issues,code_repo_open_pull_issues,code_repo_stars,code_repo_subscribers,...,reddit_posts_per_day,reddit_posts_per_hour,reddit_subscribers,total_page_views,trades_page_views,twitter_favourites,twitter_followers,twitter_following,twitter_lists,twitter_statuses
time,,,,,,,,,,,,,,,,,,,,,
2019-12-16 10:00:00,1052737,8776837,4368,12021,829,24795,765,322,41571,3505,...,73.30,3.05,1215529,47766522,767349,1000,844049,165,6631,20316
2019-12-16 09:00:00,1052733,8776769,4368,12021,829,24794,765,321,41570,3505,...,73.21,3.05,1215492,47765916,767344,1000,844049,165,6631,20316
2019-12-16 08:00:00,1052718,8776654,4368,12021,829,24794,765,321,41569,3505,...,71.29,2.97,1215465,47765308,767339,1000,844049,165,6631,20316
2019-12-16 07:00:00,1052712,8776584,4368,12021,829,24794,765,321,41568,3505,...,72.62,3.03,1215443,47764764,767335,1000,844049,165,6631,20316
2019-12-16 06:00:00,1052707,8776532,4368,12021,829,24793,765,321,41567,3505,...,71.98,3.00,1215371,47764243,767333,1000,844049,165,6631,20316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-01 04:00:00,948137,7204403,5124,11678,0,25160,903,333,43883,4190,...,116.45,4.85,1008438,38674323,666290,1000,844049,165,6631,20316
2019-01-01 03:00:00,948134,7203605,5124,11678,0,25160,903,333,43883,4190,...,118.01,4.92,1008427,38673072,666287,1000,844049,165,6631,20316
2019-01-01 02:00:00,948129,7202777,5124,11678,0,25160,903,333,43885,4190,...,119.36,4.97,1008414,38671735,666280,1000,844049,165,6631,20316


In [15]:
btc_soc.drop(["code_repo_stars","reddit_comments_per_day","reddit_posts_per_day"],axis=1,inplace=True)
btc_soc.head()

,analysis_page_views,charts_page_views,code_repo_closed_issues,code_repo_closed_pull_issues,code_repo_contributors,code_repo_forks,code_repo_open_issues,code_repo_open_pull_issues,code_repo_subscribers,comments,...,reddit_comments_per_hour,reddit_posts_per_hour,reddit_subscribers,total_page_views,trades_page_views,twitter_favourites,twitter_followers,twitter_following,twitter_lists,twitter_statuses
time,,,,,,,,,,,,,,,,,,,,,
2019-12-16 10:00:00,1052737,8776837,4368,12021,829,24795,765,322,3505,311436,...,83.74,3.05,1215529,47766522,767349,1000,844049,165,6631,20316
2019-12-16 09:00:00,1052733,8776769,4368,12021,829,24794,765,321,3505,311428,...,57.40,3.05,1215492,47765916,767344,1000,844049,165,6631,20316
2019-12-16 08:00:00,1052718,8776654,4368,12021,829,24794,765,321,3505,311426,...,71.39,2.97,1215465,47765308,767339,1000,844049,165,6631,20316
2019-12-16 07:00:00,1052712,8776584,4368,12021,829,24794,765,321,3505,311425,...,45.85,3.03,1215443,47764764,767335,1000,844049,165,6631,20316
2019-12-16 06:00:00,1052707,8776532,4368,12021,829,24793,765,321,3505,311422,...,52.67,3.00,1215371,47764243,767333,1000,844049,165,6631,20316


### MERGE

In [16]:
btc = btc_price.merge(btc_soc, on="time",how="inner")

In [17]:
btc

,high,low,open,volumefrom,volumeto,close,analysis_page_views,charts_page_views,code_repo_closed_issues,code_repo_closed_pull_issues,...,reddit_comments_per_hour,reddit_posts_per_hour,reddit_subscribers,total_page_views,trades_page_views,twitter_favourites,twitter_followers,twitter_following,twitter_lists,twitter_statuses
time,,,,,,,,,,,,,,,,,,,,,
2019-12-16 10:00:00,6343.81,6336.91,6341.45,108.28,686535.97,6339.78,1052737,8776837,4368,12021,...,83.74,3.05,1215529,47766522,767349,1000,844049,165,6631,20316
2019-12-16 09:00:00,6352.29,6336.02,6346.54,368.97,2340380.72,6341.45,1052733,8776769,4368,12021,...,57.40,3.05,1215492,47765916,767344,1000,844049,165,6631,20316
2019-12-16 08:00:00,6354.45,6332.84,6342.16,349.59,2217356.03,6346.54,1052718,8776654,4368,12021,...,71.39,2.97,1215465,47765308,767339,1000,844049,165,6631,20316
2019-12-16 07:00:00,6362.94,6339.12,6356.61,232.95,1479130.90,6342.16,1052712,8776584,4368,12021,...,45.85,3.03,1215443,47764764,767335,1000,844049,165,6631,20316
2019-12-16 06:00:00,6359.02,6344.38,6344.85,92.17,585764.06,6356.61,1052707,8776532,4368,12021,...,52.67,3.00,1215371,47764243,767333,1000,844049,165,6631,20316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-01 04:00:00,2476.01,2446.15,2454.30,570.27,1404779.38,2474.33,0,0,0,0,...,0.00,0.00,0,0,0,0,0,0,0,0
2017-08-01 03:00:00,2460.81,2443.09,2452.05,302.93,740809.94,2454.30,0,0,0,0,...,0.00,0.00,0,0,0,0,0,0,0,0
2017-08-01 02:00:00,2455.71,2445.13,2446.91,297.38,727380.07,2452.05,0,0,0,0,...,0.00,0.00,0,0,0,0,0,0,0,0


In [18]:
btc.to_csv("BTC-hourlyTS.csv",index=True)

### BITCOIN TWEETS

In [19]:
#reconvert start_date and end_date into datetime with 1 month-before lag for start
from_date = datetime.fromtimestamp(start_date)
from_date = (from_date - dateutil.relativedelta.relativedelta(months=2))
to_date = datetime.fromtimestamp(end_date)

print(from_date)
print(to_date)

2017-06-01 00:00:00
2019-12-16 10:22:48


In [20]:
test_holder = []

In [21]:
#month 1 (months adjusted)
test_from_date = from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (from_date + dateutil.relativedelta.relativedelta(months=2))
test_holder.append(get_df_tweets(test_from_date, test_to_date))

print(test_from_date)
print(test_to_date)

2017-07-01 00:00:00
2017-08-01 00:00:00


In [22]:
#month 2
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2017-08-01 00:00:00
2017-09-01 00:00:00


In [23]:
#month 3
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2017-09-01 00:00:00
2017-10-01 00:00:00


In [24]:
#month 4
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2017-10-01 00:00:00
2017-11-01 00:00:00


In [25]:
#month 5
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2017-11-01 00:00:00
2017-12-01 00:00:00


In [26]:
#month 6
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2017-12-01 00:00:00
2018-01-01 00:00:00


In [27]:
#month 7
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2018-01-01 00:00:00
2018-02-01 00:00:00


In [28]:
#month 8
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2018-02-01 00:00:00
2018-03-01 00:00:00


In [29]:
#month 9
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2018-03-01 00:00:00
2018-04-01 00:00:00


In [30]:
#month 10
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2018-04-01 00:00:00
2018-05-01 00:00:00


In [31]:
#month 11
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2018-05-01 00:00:00
2018-06-01 00:00:00


In [32]:
#month 12
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2018-06-01 00:00:00
2018-07-01 00:00:00


In [33]:
#month 13
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2018-07-01 00:00:00
2018-08-01 00:00:00


In [34]:
#month 14
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2018-08-01 00:00:00
2018-09-01 00:00:00


In [35]:
#month 15
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2018-09-01 00:00:00
2018-10-01 00:00:00


In [36]:
#month 16
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2018-10-01 00:00:00
2018-11-01 00:00:00


In [37]:
#month 17
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2018-11-01 00:00:00
2018-12-01 00:00:00


In [38]:
#month 18
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2018-12-01 00:00:00
2019-01-01 00:00:00


In [41]:
#month 19
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2019-01-01 00:00:00
2019-02-01 00:00:00


In [42]:
#month 20
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2019-02-01 00:00:00
2019-03-01 00:00:00


In [43]:
#month 21
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2019-03-01 00:00:00
2019-04-01 00:00:00


In [44]:
#month 22
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2019-04-01 00:00:00
2019-05-01 00:00:00


In [45]:
#month 23
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2019-05-01 00:00:00
2019-06-01 00:00:00


In [46]:
#month 24
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2019-06-01 00:00:00
2019-07-01 00:00:00


In [47]:
#month 25
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2019-07-01 00:00:00
2019-08-01 00:00:00


In [48]:
#month 26
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2019-08-01 00:00:00
2019-09-01 00:00:00


In [49]:
#month 27
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2019-09-01 00:00:00
2019-10-01 00:00:00


In [50]:
#month 28
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2019-10-01 00:00:00
2019-11-01 00:00:00


In [53]:
#month 29
test_from_date = test_from_date + dateutil.relativedelta.relativedelta(months=1)

test_to_date = (test_from_date + dateutil.relativedelta.relativedelta(months=1))
test_holder.append(get_df_tweets(test_from_date, test_to_date))
print(test_from_date)
print(test_to_date)

2019-11-01 00:00:00
2019-12-01 00:00:00


In [54]:
btc_df_tweet = pd.concat(test_holder, axis = 0)
btc_df_tweet.reset_index(drop=True, inplace=True)

In [55]:
btc_df_tweet

,time,text,retweets
0,2017-07-31 23:00:00,Just bought my first bit of #bitcoin. Hoping f...,0
1,2017-07-31 22:00:00,Bitcoin has maintained a bearish trend in the ...,1
2,2017-07-31 22:00:00,"Bitcoin Price Dominates as Altcoin, Ethereum P...",0
3,2017-07-31 21:00:00,"#bitcoin Price Dominates as Altcoin, #ethereum...",0
4,2017-07-31 21:00:00,"Battle of the Bitcoin, broadband competition a...",0
...,...,...,...
106106,2019-11-02 01:00:00,Interesting developments in Bitcoin: - After a...,20
106107,2019-11-02 00:00:00,It is 9252! What will be the next trend? #bitcoin,0
106108,2019-11-02 00:00:00,Yep that’s the situation for most. I’m not dis...,0
106109,2019-11-02 00:00:00,"Cryptocurrency Types, Mining, Trading, Trend: ...",0


In [56]:
btc_df_tweet.to_csv("btc_tweets_FINAL.csv",index=False)